# Exercice 03 

but analyse données du COVID19 - Allemagne

## Présentation des données

Le jeu de données pour cet exercice est composé de deux fichiers :

- covid_de.csv :
  - state : nom de l'état.
  - county : arrondissement.
  - age_group : tranche d'âge (00-04, 05-14, 15-34, 35-59, 60-79, 80-99).
  - gender : sexe (M -> homme, F -> femme).
  - date : date de relevé.
  - cases : nombre de cas.
  - deaths : nombre de morts.
  - recovered : nombre de guérisons.
  
- demographics_de.csv :
  - state : nom de l'état.
  - gender : sexe (male -> homme, femelle -> femme).
  - age_group : tranche d'âge (00-04, 05-14, 15-34, 35-59, 60-79, 80-99).
  - population : total du nombre de personnes du groupe.

Les données sont disponibles sur le site [kaggle.com](https://www.kaggle.com/headsortails/covid19-tracking-germany/).

## Exercices : partie 1

1. Récupérer le nombre d'entrées dans le fichier covid19_de.csv.
2. Récupérer le nomdre d'arrondissements (county).
3. Récupérer le nombre d'états (state).
4. Collecter le nombre d'arrondissements par état.
Afficher pour chaque état son nombre d'arrondissements.
5. Récupérer le nombre de cas positifs par état
6. Récupérer le nombre de morts par état.
7. Récupérer le nombre de cas positifs par état au 1er avril (inclus).
8. Récupérer le nombre de morts par tranche d'âge.
9. Calculer le rapport du nombre de morts par tranche d'age / nombre de morts total.
10. Récupérer le nombre de morts par sexe.
11. Calculer le rapport du nombre de morts par sexe / nombre de morts total.
12. Calculer le rapport du nombre de morts / population pour chaque région.

BONUS pour les questions 9, 11 et 12: afficher le résultat dans la table sous forme de fraction et de pourcentage   


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlf

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('../data/covid_de.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- date: date (nullable = true)
 |-- cases: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- recovered: integer (nullable = true)



In [5]:
# 1. Récupérer le nombre d'entrées dans le fichier covid19_de.csv.
df.count()

94055

In [8]:
# 2. Récupérer le nomdre d'arrondissements (county).
df.select('county').distinct().count()

412

In [9]:
# 3. Récupérer le nombre d'états (state).
df.select('state').distinct().count()

16

In [25]:
# 4. Collecter le nombre d'arrondissements par état. Afficher pour chaque état son nombre d'arrondissements.
df.groupby('state')\
  .agg(sqlf.count_distinct('county').alias('nb_de_couties_per_state'))\
  .show()

+--------------------+-----------------------+
|               state|nb_de_couties_per_state|
+--------------------+-----------------------+
|      Sachsen-Anhalt|                     14|
|       Niedersachsen|                     45|
|         Brandenburg|                     18|
|              Berlin|                     12|
|              Bayern|                     96|
|             Sachsen|                     13|
|             Hamburg|                      1|
|              Bremen|                      2|
| Nordrhein-Westfalen|                     53|
|  Baden-Wuerttemberg|                     44|
|              Hessen|                     26|
|Mecklenburg-Vorpo...|                      8|
|            Saarland|                      6|
|  Schleswig-Holstein|                     15|
|          Thueringen|                     23|
|     Rheinland-Pfalz|                     36|
+--------------------+-----------------------+



In [30]:
# 5. Récupérer le nombre de cas positifs par état
df.groupby('state')\
  .agg(sqlf.sum('cases').alias('nb_cases'))\
  .orderBy(sqlf.col('nb_cases').desc())\
  .show()

+--------------------+--------+
|               state|nb_cases|
+--------------------+--------+
|              Bayern|   50048|
| Nordrhein-Westfalen|   46613|
|  Baden-Wuerttemberg|   36565|
|       Niedersachsen|   14132|
|              Hessen|   11534|
|              Berlin|    8860|
|     Rheinland-Pfalz|    7374|
|             Sachsen|    5503|
|             Hamburg|    5250|
|         Brandenburg|    3510|
|          Thueringen|    3327|
|  Schleswig-Holstein|    3292|
|            Saarland|    2840|
|      Sachsen-Anhalt|    1973|
|              Bremen|    1727|
|Mecklenburg-Vorpo...|     815|
+--------------------+--------+



In [32]:
# 6. Récupérer le nombre de morts par état.
df.groupby('state')\
  .agg(sqlf.sum('deaths').alias('nb_deaths'))\
  .orderBy(sqlf.col('nb_deaths').desc())\
  .show()

+--------------------+---------+
|               state|nb_deaths|
+--------------------+---------+
|              Bayern|     2618|
|  Baden-Wuerttemberg|     1838|
| Nordrhein-Westfalen|     1721|
|       Niedersachsen|      647|
|              Hessen|      516|
|             Hamburg|      261|
|     Rheinland-Pfalz|      238|
|             Sachsen|      225|
|              Berlin|      222|
|          Thueringen|      182|
|            Saarland|      174|
|         Brandenburg|      167|
|  Schleswig-Holstein|      155|
|      Sachsen-Anhalt|       62|
|              Bremen|       55|
|Mecklenburg-Vorpo...|       20|
+--------------------+---------+



In [44]:
# 7. Récupérer le nombre de cas positifs par état au 1er avril (inclus).
df.filter(sqlf.col('date') <= '2020-04-01')\
  .groupby('state')\
  .agg(sqlf.sum('cases').alias('nb_cases_as_of_april_1st'))\
  .orderBy(sqlf.col('nb_cases_as_of_april_1st').desc())\
  .show()

+--------------------+------------------------+
|               state|nb_cases_as_of_april_1st|
+--------------------+------------------------+
|              Bayern|                   20234|
| Nordrhein-Westfalen|                   16818|
|  Baden-Wuerttemberg|                   16582|
|       Niedersachsen|                    5168|
|              Hessen|                    3905|
|     Rheinland-Pfalz|                    3321|
|              Berlin|                    3088|
|             Hamburg|                    2715|
|             Sachsen|                    2473|
|  Schleswig-Holstein|                    1377|
|         Brandenburg|                    1125|
|            Saarland|                    1086|
|          Thueringen|                     944|
|      Sachsen-Anhalt|                     815|
|Mecklenburg-Vorpo...|                     448|
|              Bremen|                     328|
+--------------------+------------------------+



In [45]:
# 8. Récupérer le nombre de morts par tranche d'âge.
df.groupby('age_group')\
  .agg(sqlf.sum('deaths').alias('nb_deaths'))\
  .show()

+---------+---------+
|age_group|nb_deaths|
+---------+---------+
|    80-99|     5733|
|       NA|        0|
|    05-14|        0|
|    35-59|      418|
|    60-79|     2931|
|    00-04|        1|
|    15-34|       18|
+---------+---------+



In [100]:
# 9. Calculer le rapport du nombre de morts par tranche d'age / nombre de morts total.
# BONUS: afficher le résultat dans la table sous forme de fraction et de pourcentage
total_deaths = df.agg(sqlf.sum('deaths')).collect()[0][0]
df.groupby('age_group')\
  .agg(sqlf.sum('deaths').alias('nb_deaths'))\
  .withColumn('frac_ratio_to_total',
             sqlf.round(
                 sqlf.col('nb_deaths') / total_deaths,
                 4
             ))\
  .withColumn('perc_ratio_to_total', 
              sqlf.format_string('%.4f%%', sqlf.col('nb_deaths') / total_deaths * 100)
             )\
  .show()

+---------+---------+-------------------+-------------------+
|age_group|nb_deaths|frac_ratio_to_total|perc_ratio_to_total|
+---------+---------+-------------------+-------------------+
|    80-99|     5733|             0.6299|           62.9931%|
|       NA|        0|                0.0|            0.0000%|
|    05-14|        0|                0.0|            0.0000%|
|    35-59|      418|             0.0459|            4.5929%|
|    60-79|     2931|             0.3221|           32.2053%|
|    00-04|        1|             1.0E-4|            0.0110%|
|    15-34|       18|              0.002|            0.1978%|
+---------+---------+-------------------+-------------------+



In [56]:
# 10. Récupérer le nombre de morts par sexe.
df.groupby('gender')\
  .agg(sqlf.sum('deaths').alias('nb_deaths'))\
  .show()

+------+---------+
|gender|nb_deaths|
+------+---------+
|     F|     4066|
|    NA|        5|
|     M|     5030|
+------+---------+



In [101]:
# 11. Calculer le rapport du nombre de morts par sexe / nombre de morts total.
# BONUS: afficher le résultat dans la table sous forme de fraction et de pourcentage
total_deaths = df.agg(sqlf.sum('deaths')).collect()[0][0]
df.groupby('gender')\
  .agg(sqlf.sum('deaths').alias('nb_deaths'))\
  .withColumn('frac_ratio_to_total',
             sqlf.round(
                 sqlf.col('nb_deaths') / total_deaths,
                 3
             ))\
  .withColumn('perc_ratio_to_total',
             sqlf.format_string('%.3f%%',
                 sqlf.col('nb_deaths') / total_deaths * 100,
             ))\
  .show()

+------+---------+-------------------+-------------------+
|gender|nb_deaths|frac_ratio_to_total|perc_ratio_to_total|
+------+---------+-------------------+-------------------+
|     F|     4066|              0.447|            44.676%|
|    NA|        5|              0.001|             0.055%|
|     M|     5030|              0.553|            55.269%|
+------+---------+-------------------+-------------------+



In [102]:
# 12. Calculer le rapport du nombre de morts / population pour chaque région.
# BONUS: afficher le résultat dans la table sous forme de fraction et de pourcentage
df_population = spark.read.csv('../data/demographics_de.csv', header=True, inferSchema=True)
df_population.groupby('state')\
  .agg(sqlf.sum('population').alias('state_population'))\
  .sort(sqlf.col('state_population').desc())\
  .join(
    df.groupby('state')\
      .agg(sqlf.sum('deaths').alias('nb_deaths')),
    on='state')\
  .withColumn('frac_ratio_deaths_to_population', sqlf.round(
                sqlf.col('nb_deaths') / sqlf.col('state_population'),
                8
              )
             )\
  .withColumn('perc_ratio_deaths_to_population', sqlf.format_string('%.4f%%', 
                sqlf.col('nb_deaths') / sqlf.col('state_population') * 100
              )
             )\
  .show()

+--------------------+----------------+---------+-------------------------------+-------------------------------+
|               state|state_population|nb_deaths|frac_ratio_deaths_to_population|perc_ratio_deaths_to_population|
+--------------------+----------------+---------+-------------------------------+-------------------------------+
|       Niedersachsen|         7982448|      647|                       8.105E-5|                        0.0081%|
|         Brandenburg|         2511917|      167|                       6.648E-5|                        0.0066%|
|              Berlin|         3644826|      222|                       6.091E-5|                        0.0061%|
|              Bayern|        13076721|     2618|                       2.002E-4|                        0.0200%|
|             Hamburg|         1841179|      261|                      1.4176E-4|                        0.0142%|
|              Bremen|          682986|       55|                       8.053E-5|       